## Organizando as imagens em uma pasta única

In [10]:
import os
import shutil
from tqdm import tqdm

# Diretório principal contendo as pastas com as fotos
diretorio_principal = './archive/lfw-funneled/lfw_funneled'

# Diretório onde as fotos serão copiadas
diretorio_destino = 'fotos_celebs'

# Cria o diretório de destino se ele não existir
os.makedirs(diretorio_destino, exist_ok=True)
arquivos_jpg = []

# Percorre todas as pastas e arquivos dentro do diretório principal
for root, dirs, files in os.walk(diretorio_principal):
    for file in files:
        # Verifica se o arquivo tem a extensão .jpg
        if file.lower().endswith('.jpg'):
            # Adiciona o caminho completo do arquivo à lista
            arquivos_jpg.append(os.path.join(root, file))

# Exibe a barra de progresso para copiar os arquivos
for caminho_arquivo in tqdm(arquivos_jpg, desc="Copiando fotos", unit="arquivo"):
    # Copia o arquivo para o diretório de destino
    shutil.copy(caminho_arquivo, diretorio_destino)

print("Todas as fotos .jpg foram copiadas para a pasta 'fotos_celebs'.")

Copiando fotos: 100%|██████████| 13233/13233 [01:48<00:00, 121.83arquivo/s]

Todas as fotos .jpg foram copiadas para a pasta 'fotos_celebs'.


## Criando função para converter imagens em embeddings

In [1]:
import os
import torch
import torchvision.transforms as transforms
import torch.nn as nn
from torchvision import models
from PIL import Image
import numpy as np
from tqdm import tqdm

# Diretório contendo as imagens
diretorio_imagens = 'fotos_celebs'

# Arquivo para salvar os embeddings
arquivo_embeddings = 'embeddings.npy'

# Configura o modelo e a transformação das imagens
class ResNet50WithEmbedding(nn.Module):
    def __init__(self, embedding_dim):
        super(ResNet50WithEmbedding, self).__init__()
        self.resnet50 = models.resnet50(pretrained=True)
        self.resnet50.fc = nn.Linear(self.resnet50.fc.in_features, embedding_dim)
    
    def forward(self, x):
        return self.resnet50(x)

embedding_dim = 512
model = ResNet50WithEmbedding(embedding_dim)
model.eval()  # Configura o modelo para modo de avaliação

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Função para converter imagem em embedding
def imagem_para_embedding(caminho_imagem):
    image = Image.open(caminho_imagem).convert('RGB')
    image = transform(image).unsqueeze(0)  # Adiciona a dimensão do lote
    with torch.no_grad():
        features = model(image)
    return features.squeeze().numpy()  # Remove a dimensão do lote e converte para numpy array

c:\Users\felip\anaconda3\envs\yolo_pose\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\felip\anaconda3\envs\yolo_pose\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Testando em uma imagem para ver como está funcionando

In [2]:
caminho_imagem = 'fotos_celebs/Aaron_Eckhart_0001.jpg'

image = Image.open(caminho_imagem).convert('RGB')
image = transform(image).unsqueeze(0)  # Adiciona a dimensão do lote
with torch.no_grad():
    features = model(image)
embedding = features.squeeze().numpy()

In [3]:
embedding.shape

(512,)

## Convertendo todas as imagens em embeddings

In [4]:
# Lista os arquivos de imagem
arquivos_imagens = [os.path.join(diretorio_imagens, f) for f in os.listdir(diretorio_imagens) if f.lower().endswith('.jpg')]

# Armazena os embeddings
embeddings = []

# Processa as imagens e gera embeddings
for caminho_imagem in tqdm(arquivos_imagens, desc="Gerando embeddings", unit="imagem"):
    embedding = imagem_para_embedding(caminho_imagem)
    embeddings.append(embedding)

# Salva os embeddings em um arquivo .npy
np.save(arquivo_embeddings, np.array(embeddings))

print(f"Os embeddings foram salvos em '{arquivo_embeddings}'.")

Gerando embeddings: 100%|██████████| 13233/13233 [38:07<00:00,  5.78imagem/s] 

Os embeddings foram salvos em 'embeddings.npy'.


In [14]:
import pandas as pd

df = pd.DataFrame([arquivos_imagens, embeddings]).T
df = df.rename(columns={0: 'filename', 1: 'emb'})
df['filename'] = df['filename'].str.replace('fotos_celebs\\', '')
df.to_parquet('embeddings.parquet')